<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
!pip install keras_bert

In [2]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-447s7e4j
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-447s7e4j
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=545a1bb796fff1508604ed7cd2b6b2cf56d22d17b17564b93d638f10ce4ecc6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_kzbp0wn/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [4]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

fatal: destination path 'bachelor_design' already exists and is not an empty directory.
/content/bachelor_design


In [5]:
!git checkout equal_realize
!git pull

Already on 'equal_realize'
Your branch is up to date with 'origin/equal_realize'.
Already up to date.


# Mount GDriver
Perpare Colab environment.

In [6]:
from driver_amount import addh

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
[Colab] Using address head: /gdrive/My Drive


# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [8]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

Using TensorFlow backend.


In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [10]:
tag_vocab

{'': 0,
 'B-EI': 4,
 'B-EIF': 10,
 'B-EO': 2,
 'B-EQ': 8,
 'B-ILF': 6,
 'I-EI': 5,
 'I-EIF': 11,
 'I-EO': 3,
 'I-EQ': 9,
 'I-ILF': 7,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [12]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [15]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [16]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [17]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
crf_1 (CRF)                     (None, 512, 12)      9396        model_2[1][0]              

# Fine-tuning

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_LOG_PATH,
    1
    )

In [20]:
model.fit(
    x=[token_id_seqs, segment_seqs],
    y=[tag_id_seqs], 
    batch_size=4, 
    epochs=15,
    verbose=1,
    validation_split=0.1,
    callbacks=[epoch_callback]
)



Train on 7005 samples, validate on 779 samples
2020-03-20 01:27:19,371-[INFO] Start Train. Checkpoint will be save at /gdrive/My Drive/Graduation/Model/model.h5
2020-03-20 01:27:19,372-[INFO] Detail log file will be saved at /gdrive/My Drive/Graduation/Model/train.log
Epoch 1/5


KeyboardInterrupt: ignored

In [0]:
model.save(addh + config.MODEL_PATH)